In [19]:
import pandas as pd
import scipy

In [2]:
data1 = scipy.io.loadmat("D:\Research Dr. Power\BCI_IV_2a\BCI_IV_2a\A01E.mat")

In [6]:
data1.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data'])

In [12]:
data1['data'].shape

(1, 9)

In [40]:
data1['data'][0][4][0][0][0][0]

array([-3.27148438, -3.95507812, -1.51367188,  0.68359375,  1.02539062,
        0.68359375,  3.95507812,  2.34375   ,  7.32421875,  7.76367188,
        6.15234375,  3.95507812,  2.34375   ,  9.375     , 10.10742188,
       10.15625   ,  8.10546875,  6.00585938, 10.15625   ,  9.03320312,
        8.54492188, 10.05859375, 16.6015625 ,  6.34765625, -0.48828125])